# Chapter 3: GPU programming.

This chapter and the next will make extensive use of GPUs. Sadly, depending on your machine, it can be impossible to use it in python. For example, at the time I do this tutorial, Radeon is not supported. Also some of your laptop may not have GPUs. These reasons pushed me to run the next Chapter on google collab (https://colab.research.google.com/). If you are still interested in using your own GPUs here are some advice/links that might help you:

- https://towardsdatascience.com/installing-tensorflow-with-cuda-cudnn-and-gpu-support-on-windows-10-60693e46e781
- https://www.youtube.com/watch?v=hHWkvEcDBO0
- https://www.youtube.com/watch?v=KZFn0dvPZUQ
- https://towardsdatascience.com/installing-tensorflow-gpu-in-ubuntu-20-04-4ee3ca4cb75d
- https://medium.com/analytics-vidhya/install-tensorflow-2-for-amd-gpus-87e8d7aeb812

Why do we want to use GPUs ?

GPUs hardware is designed for data parallelism. Maximum throughput is achieved when you are computing the same operations on many different elements at once.

https://docs.nvidia.com/cuda/cuda-c-programming-guide/index.html

Among the tasks that do significantly benefit from parallel processing is deep learning. Some tasks can't be used in parallel (When you need to have the same object in memory, e.g calculating a series like fibonnaci).

One thing that could be nice would be to write the same code as normal (numpy, pandas,..) but just to run computation on a GPU. This would make it easier to parallelize processes. Some companies/university/people are working on this kind of libraries and that's what we are going to use in this section.

Structure:
- [Collab](#Collab)
- [CuPy](#CuPy)
- [Numba](#Numba)
- [CuDF and CuML](#CuDF)
- [TODO](#TODO)

<a name="Collab"></a>
## Google Collab

Stockage is limited to 60 gb (see on the left) 

Ram is limited to 12 gb (top right)

You can select gpu accelerated from modify->parameter of the notebook. 

Create text block and code block

You can create section.

Resembles jupyter notebook and uses ipynb.

Change color and shortcut in utils

The os you are connected to is ubuntu
To run something in the terminal you need to add "!" in front of it

Python already installed.

Session are limited in time.

To use GPU go to Execution and modify the type of execution

In [ ]:
# Check Python Version
!python --version

Python 3.7.11


In [ ]:
# Check Ubuntu Version
!lsb_release -a

In [ ]:
# Check CUDA/cuDNN Version
!nvcc -V && which nvcc

In [ ]:
# Check GPU
!nvidia-smi

In [ ]:
!cat /proc/cpuinfo

<a name="CuPy"></a>
## CuPy

CuPy is the GPU equivalent to Numpy. CuPy uses the same methods that numpy so cost entry going from Numpy to CuPy is low

In [ ]:
import cupy as cp
import numpy as np

In [ ]:
z = cp.arange(6).reshape(2, 3).astype('f')
z

array([[0., 1., 2.],
       [3., 4., 5.]], dtype=float32)

In [ ]:
z.mean(axis=0)

array([1.5, 2.5, 3.5], dtype=float32)

In [ ]:
z.sum(axis=1)

array([ 3., 12.], dtype=float32)

In [ ]:
z.dot(z.T).astype('int')

array([[ 5, 14],
       [14, 50]])

In [ ]:
ary = cp.arange(10).reshape((2,5))
print(repr(ary))
print(ary.dtype)
print(ary.shape)
print(ary.strides)
print(ary.device)

array([[0, 1, 2, 3, 4],
       [5, 6, 7, 8, 9]])
int64
(2, 5)
(40, 8)
<CUDA Device 0>


In [ ]:
ary_cpu = np.arange(10)
ary_gpu = cp.asarray(ary_cpu)
print('cpu:', ary_cpu)
print('gpu:', ary_gpu)

cpu: [0 1 2 3 4 5 6 7 8 9]
gpu: [0 1 2 3 4 5 6 7 8 9]


In [ ]:
ary_cpu_returned = cp.asnumpy(ary_gpu)
print(repr(ary_cpu_returned))
print(type(ary_cpu_returned))

array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9])
<class 'numpy.ndarray'>


In [ ]:
print(ary_gpu * 2)
print(cp.exp(-0.5 * ary_gpu**2))
print(cp.linalg.norm(ary_gpu))
print(cp.random.normal(loc=5, scale=2.0, size=10))

[ 0  2  4  6  8 10 12 14 16 18]
[1.00000000e+00 6.06530660e-01 1.35335283e-01 1.11089965e-02
 3.35462628e-04 3.72665317e-06 1.52299797e-08 2.28973485e-11
 1.26641655e-14 2.57675711e-18]
16.881943016134134
[4.33906114 4.67377457 1.08905314 9.21676987 6.70516547 3.11660347
 3.57749795 2.01394103 6.62182583 5.56588289]


You may notice a slight pause when you run these functions the first time. This is because CuPy has to compile the CUDA functions on the fly, and then cache them to disk for reuse in the future. Let's compare some performance

In [ ]:
import pandas as pd
import cupy as cp
import numpy as np

Let's compare a simple multiplication:

In [ ]:
%%timeit 
# small example taken from here https://giters.com/cupy/cupy/issues/4891?amp=1

a_cpu = np.ones((1000, 20000), dtype='float32')
b_cpu = np.ones((20000, 2000), dtype='float32')
z_cpu = np.matmul(a_cpu, b_cpu)

1 loop, best of 5: 1.2 s per loop


In [ ]:
%%timeit 
# small example taken from here https://giters.com/cupy/cupy/issues/4891?amp=1

a_gpu = cp.ones((1000, 20000), dtype='float32')
b_gpu = cp.ones((20000, 2000), dtype='float32')
z_gpu = cp.matmul(a_gpu, b_gpu)

The slowest run took 3125.66 times longer than the fastest. This could mean that an intermediate result is being cached.
1 loop, best of 5: 221 µs per loop


Now the analytical solution of OLS:

In [ ]:
X_cpu = np.random.rand(20000, 1000).astype('f')
Y_cpu = np.random.rand(20000, 1).astype('f')

In [ ]:
X_gpu = cp.asarray(X_cpu,dtype='float32')
Y_gpu = cp.asarray(Y_cpu,dtype='float32')

In [ ]:
%%timeit 
beta = np.matmul(np.linalg.inv(np.matmul(X_cpu.T,X_cpu)),np.matmul(X_cpu.T,Y_cpu))

1 loop, best of 5: 483 ms per loop


In [ ]:
%%timeit 
beta = np.matmul(np.linalg.inv(np.matmul(X_gpu.T,X_gpu)),np.matmul(X_gpu.T,Y_gpu))

10 loops, best of 5: 55.6 ms per loop


This does not mean that GPUs are always faster. When are they worst ? 
Read more here https://towardsdatascience.com/heres-how-to-use-cupy-to-make-numpy-700x-faster-4b920dda1f56

In [ ]:
df = pd.read_csv("sample_data/california_housing_train.csv")
intercept = np.ones(len(df))

y = np.array(df["median_house_value"],dtype='float32')
X = np.array(df.drop(["median_house_value"],axis=1),dtype='float32')

In [ ]:
y_gpu = cp.asarray(y,dtype='float32')
X_gpu = cp.asarray(X,dtype='float32')

In [ ]:
%%timeit 
beta = np.matmul(np.linalg.inv(np.matmul(X.T,X)),np.matmul(X.T,y))

1000 loops, best of 5: 412 µs per loop


In [ ]:
%%timeit 
beta = cp.matmul(cp.linalg.inv(cp.matmul(X_gpu.T,X_gpu)),cp.matmul(X_gpu.T,y_gpu))

The slowest run took 6.37 times longer than the fastest. This could mean that an intermediate result is being cached.
1000 loops, best of 5: 761 µs per loop


Also cupy works best when using ufunc but like we have seen in tthe introduction, not every operation can be done using ufunc. To overcome this issue you can create your own "Kernel". This Kernel is written in C++ so don't expect to understand everything directly. (read more here https://docs.cupy.dev/en/stable/user_guide/kernel.html)

In [ ]:
# Example taken from docs

x = cp.arange(10, dtype=np.float32).reshape(2, 5)
y = cp.arange(5, dtype=np.float32)

add_reverse = cp.ElementwiseKernel(
    'T x, raw T y', 
    'T z',
    '''
    z = x + y[_ind.size() - i - 1];
    ''',
    'add_reverse')

In [ ]:
add_reverse(x,y)

array([[ 2.1311454e-01, -1.0437331e+27,  1.8231753e+00, -1.1218646e+17,
         4.2748837e+00],
       [ 9.0000000e+00,  9.0000000e+00,  9.0000000e+00,  9.0000000e+00,
         9.0000000e+00]], dtype=float32)

You can find even more complex custom CUDA kernel. 

<a name="Numba"></a>
## Numba

Numba is a just-in-time (https://en.wikipedia.org/wiki/Just-in-time_compilation), type-specializing, function compiler for accelerating numerically-focused Python. That's a long list, so let's break down those terms: 

- function compiler: Numba compiles Python functions, not entire applications, and not parts of functions. Numba does not replace your Python interpreter, but is just another Python module that can turn a function into a (usually) faster function.
- type-specializing: Numba speeds up your function by generating a specialized implementation for the specific data types you are using. Python functions are designed to operate on generic data types, which makes them very flexible, but also very slow. In practice, you only will call a function with a small number of argument types, so Numba will generate a fast implementation for each set of types.
- just-in-time: Numba translates functions when they are first called. This ensures the compiler knows what argument types you will be using. This also allows Numba to be used interactively in a Jupyter notebook just as easily as a traditional application


In [ ]:
from numba import jit
import math

@jit
def hypot(x, y):
    # Implementation from https://en.wikipedia.org/wiki/Hypot
    x = abs(x);
    y = abs(y);
    t = min(x, y);
    x = max(x, y);
    t = t / x;
    return x * math.sqrt(1+t*t)

The first time we call hypot, the compiler is triggered and compiles a machine code implementation for float inputs. Numba also saves the original Python implementation of the function in the .py_func attribute, so we can call the original Python code to make sure we get the same answer:

In [ ]:
%timeit hypot(10,20)

The slowest run took 729428.58 times longer than the fastest. This could mean that an intermediate result is being cached.
1 loop, best of 5: 327 ns per loop


In [ ]:
%timeit hypot.py_func(10,20)

The slowest run took 15.48 times longer than the fastest. This could mean that an intermediate result is being cached.
1000000 loops, best of 5: 946 ns per loop


How does numba works ? From https://towardsdatascience.com/speed-up-your-algorithms-part-2-numba-293e554c5cc1

![numba](img/numba.png)

Numba can also be used to create new numpy universal function.
https://numba.pydata.org/numba-doc/dev/user/vectorize.html

Numba’s vectorize allows Python functions taking scalar input arguments to be used as NumPy ufuncs. Creating a traditional NumPy ufunc is not the most straightforward process and involves writing some C code. Numba makes this easy. Using the vectorize() decorator, Numba can compile a pure Python function into a ufunc that operates over NumPy arrays as fast as traditional ufuncs written in C.

Using vectorize(), you write your function as operating over input scalars, rather than arrays. Numba will generate the surrounding loop (or kernel) allowing efficient iteration over the actual inputs. 

In [ ]:
%%timeit 

from numba import vectorize, float64

@vectorize([float64(float64, float64)])
def f(x, y):
    return x + y
N = 10000
A = np.array(np.random.sample(N), dtype=dtype)
B = np.array(np.random.sample(N), dtype=dtype)
f(A,B)

In [ ]:
%%timeit

from numba import vectorize, float64

@vectorize([float64(float64, float64)], target = "parallel")
def f(x, y):
    return x + y

N = 10000
A = np.array(np.random.sample(N), dtype=dtype)
B = np.array(np.random.sample(N), dtype=dtype)
f(A,B)

In [ ]:
import math
from numba import vectorize
import numpy as np

@vectorize(['float32(float32, float32, float32)',
            'float64(float64, float64, float64)'])
def cu_discriminant(a, b, c):
    return math.sqrt(b ** 2 - 4 * a * c)

N = 10000
dtype = np.float32

# prepare the input
A = np.array(np.random.sample(N), dtype=dtype)
B = np.array(np.random.sample(N) + 10, dtype=dtype)
C = np.array(np.random.sample(N), dtype=dtype)

D = cu_discriminant(A, B, C)

print(D)  # print result

Numba also supports GPU programming.

In [ ]:
import numba
from numba import cuda

In [ ]:
# list of devices
print(cuda.gpus)
# Select your device
numba.cuda.select_device(0)

CUDA has an execution model unlike the traditional sequential model used for programming CPUs. In CUDA, the code you write will be executed by multiple threads at once (often hundreds or thousands). Your solution will be modeled by defining a thread hierarchy of grid, blocks, and threads.

Numba also exposes three kinds of GPU memory:

* global device memory
* shared memory
* local memory

For all but the simplest algorithms, it is important that you carefully consider how to use and access memory in order to minimize bandwidth requirements and contention.

NVIDIA recommends that programmers focus on following those recommendations to achieve the best performance:

* Find ways to parallelize sequential code
* Minimize data transfers between the host and the device
* Adjust kernel launch configuration to maximize device utilization
* Ensure global memory accesses are coalesced
* Minimize redundant accesses to global memory whenever possible
* Avoid different execution paths within the same warp

https://stackoverflow.com/questions/4391162/cuda-determining-threads-per-block-blocks-per-grid
https://en.wikipedia.org/wiki/CUDA

In [ ]:
from numba import cuda
import math
import numpy as np

# CUDA kernel
@cuda.jit
def my_kernel(io_array):
    pos = cuda.grid(1)
    if pos < io_array.size:
        io_array[pos] *= 2 # do the computation

# Host code   
data = np.ones(256)
threadsperblock = 256
blockspergrid = math.ceil(data.shape[0] / threadsperblock)
my_kernel[blockspergrid, threadsperblock](data)
print(data)

In [ ]:
@cuda.jit
def matmul(A, B, C):
    """Perform matrix multiplication of C = A * B
    """
    row, col = cuda.grid(2)
    if row < C.shape[0] and col < C.shape[1]:
        tmp = 0.
        for k in range(A.shape[1]):
            tmp += A[row, k] * B[k, col]
        C[row, col] = tmp
        
# Host code

# Initialize the data arrays
A = np.full((24, 12), 3, np.float) # matrix containing all 3's
B = np.full((12, 22), 4, np.float) # matrix containing all 4's

# Copy the arrays to the device
A_global_mem = cuda.to_device(A)
B_global_mem = cuda.to_device(B)

# Allocate memory on the device for the result
C_global_mem = cuda.device_array((24, 22))

# Configure the blocks
threadsperblock = (16, 16)
blockspergrid_x = int(math.ceil(A.shape[0] / threadsperblock[0]))
blockspergrid_y = int(math.ceil(B.shape[1] / threadsperblock[1]))
blockspergrid = (blockspergrid_x, blockspergrid_y)

# Start the kernel 
matmul[blockspergrid, threadsperblock](A_global_mem, B_global_mem, C_global_mem)

# Copy the result back to the host
C = C_global_mem.copy_to_host()

print(C)

In [ ]:
from numba import vectorize
import numpy as np

@vectorize(['int64(int64, int64)'], target='cuda')
def add_ufunc(x, y):
    return x + y

In [ ]:
a = np.array([1, 2, 3, 4])
b = np.array([10, 20, 30, 40])
b_col = b[:, np.newaxis] # b as column array
c = np.arange(4*4).reshape((4,4))

print('a+b:\n', add_ufunc(a, b))
print()
print('b_col + c:\n', add_ufunc(b_col, c))

A lot of things just happened! Numba automatically:

* Compiled a CUDA kernel to execute the ufunc operation in parallel over all the input elements.
* Allocated GPU memory for the inputs and the output.
* Copied the input data to the GPU.
* Executed the CUDA kernel with the correct kernel dimensions given the input sizes.
* Copied the result back from the GPU to the CPU.
* Returned the result as a NumPy array on the host.

This is very convenient for testing, but copying data back and forth between the CPU and GPU can be slow and hurt performance. In the next tutorial notebook, you'll learn about device management, memory allocation, and using CuPy arrays with Numba.

In [ ]:
%timeit np.add(b_col, c)   # NumPy on CPU

In [ ]:
%timeit add_ufunc(b_col, c) # Numba on GPU

Why is the GPU slower ?
* Our inputs are too small: the GPU achieves performance through parallelism, operating on thousands of values at once. Our test inputs have only 4 and 16 integers, respectively. We need a much larger array to even keep the GPU busy.
* Our calculation is too simple: Sending a calculation to the GPU involves quite a bit of overhead compared to calling a function on the CPU. If our calculation does not involve enough math operations (often called "arithmetic intensity"), then the GPU will spend most of its time waiting for data to move around.
* We copy the data to and from the GPU: While including the copy time can be realistic for a single function, often we want to run several GPU operations in sequence. In those cases, it makes sense to send data to the GPU and keep it there until all of our processing is complete.
* Our data types are larger than necessary: Our example uses int64 when we probably don't need it. Scalar code using data types that are 32 and 64-bit run basically the same speed on the CPU, but 64-bit data types have a significant performance cost on the GPU. Basic arithmetic on 64-bit floats can be anywhere from 2x (Pascal-architecture Tesla) to 24x (Maxwell-architecture GeForce) slower than 32-bit floats. NumPy defaults to 64-bit data types when creating arrays, so it is important to set the dtype attribute or use the ndarray.astype() method to pick 32-bit types when you need them.

Let's see a bigger example

In [ ]:
import math  # Note that for the CUDA target, we need to use the scalar functions from the math module, not NumPy

SQRT_2PI = np.float32((2*math.pi)**0.5)  # Precompute this constant as a float32.  Numba will inline it at compile time.

@vectorize(['float32(float32, float32, float32)'], target='cuda')
def gaussian_pdf(x, mean, sigma):
    '''Compute the value of a Gaussian probability density function at x with given mean and sigma.'''
    return math.exp(-0.5 * ((x - mean) / sigma)**2) / (sigma * SQRT_2PI)

In [ ]:
x = np.random.uniform(-3, 3, size=1000000).astype(np.float32)
mean = np.float32(0.0)
sigma = np.float32(1.0)

# Quick test
gaussian_pdf(x[0], 0.0, 1.0)

In [ ]:
import scipy.stats # for definition of gaussian distribution
norm_pdf = scipy.stats.norm
%timeit norm_pdf.pdf(x, loc=mean, scale=sigma)

In [ ]:
%timeit gaussian_pdf(x, mean, sigma)

## Numba limitations

Numba accelerates your code. So why should'nt we use it for everything if it's has simple as putting a decorator in front of a function ?

Well it's not that simple.

Numba is numerically-focused: Currently, Numba is focused on numerical data types, like int, float, and complex. There is very limited string processing support, and many string use cases are not going to work well on the GPU. To get best results with Numba, you will likely be using NumPy arrays. When you run a function that uses string or dict, python ignores the jit decorator and run the function as normal.

In [ ]:
@jit()
def cannot_compile(x):
    return x['key']

cannot_compile(dict(key='hey heres your value'))

To avoid this type of behavior (we want an error message and not just a warning) we add the argument nopython = True.

In [ ]:
@jit(nopython=True)
def cannot_compile(x):
    return x['key']

cannot_compile(dict(key='hey heres your value'))

<a name="CuDF"></a>
## CuDF and CuML

CuDF is develeopped by rapidsai (https://rapids.ai/) and like CuPY the goal is to have the features of pandas using GPUs. CuML is also develeopped by rapidsai (https://rapids.ai/) and this time the library we want to apply GPUs is Scikit-learn. Installing them on google collab is a bit complex so we will directly use their google collab :https://colab.research.google.com/drive/1rY7Ln6rEE1pOlfSHCYOVaqt8OvDO35J0#forceEdit=true&sandboxMode=true&scrollTo=JI7UTXbhaBon

<a name="TODO"></a>
## TODO

code review: 
- https://www.programcreek.com/python/example/111769/cupy.ElementwiseKernel